# Исследование надежности заемщиков

**Описание проекта:**  
Наш заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение, уровень дохода и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.  

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.  

**Цели исследования:**  
- оценить, влияют ли семейное положение, уровень дохода и количество детей клиента на факт погашения кредита в срок
- предоставить данные для построения модели кредитного скоринга 

**Ход исследования:**  

Статистика о платёжеспособности клиентов хранится в файле: '/datasets/real_estate_data.csv'.  
О качестве данных ничего неизвестно, поэтому понадобится предварительный обзор данных.   
После обзора я проверю данные на аномалии, пропуски, дубликаты, правильность исходных типов и постраюсь устранить все проблемы, которые могут помешать в процессе исследования, либо привести к искажению конечного результата. Далее я добавлю необходимые категории, которые помогут в построении модели кредитного скоринга и приступлю к исследованию надёжности клиентов. 

Таким образом, исследование будет состоять из четырёх этапов:
- Обзор данных
- Предобработка данных
- Исследовательский анализ данных
- Подведение итогов исследования

## Обзор с данных

**1.1 Откроем файл с данными и сохраним его в переменную data:**

In [1]:
# импорт библиотек:
import pandas as pd

# прочитаем csv-файл и сохраним данные в переменную data:
data = pd.read_csv('C:\\Users\\Refresh\\Projects\\Bank\\data.csv')

**1.2 Выведем первые 5 строк датафрейма data на экран:**

In [2]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


**1.3 Выведем основную информацию о датафрейме с помощью метода info**():

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод:** Датафрейм data состоит из 21525 строк и 12 столбцов. В нем содержится следующая информация о заёмщиках:  
- *children* — количество детей в семье  
- *days_employed* — общий трудовой стаж в днях  
- *dob_years* — возраст клиента в годах  
- *education* — уровень образования клиента  
- *education_id* — идентификатор уровня образования  
- *family_status* — семейное положение  
- *family_status_id* — идентификатор семейного положения  
- *gender* — пол клиента  
- *income_type* — тип занятости  
- *debt* — имел ли задолженность по возврату кредитов  
- *total_income* — ежемесячный доход  
- *purpose* — цель получения кредита  

На первом этапе обнаружены следующие проблемы в исходных данных:
- аномалии в стоблце *days_employed*   
- пропущенные значения в столбцах *days_employed* и *total_income*     

Устраним пошагово эти и другие возможные проблемы, чтобы подготовить данные для исследования.

## Предобработка данных

### Обработка аномальных значений

#### Проверим уникальные значения столбца *days_employed:*

In [4]:
data['days_employed'].unique()

array([-8437.67302776, -4024.80375385, -5623.42261023, ...,
       -2113.3468877 , -3112.4817052 , -1984.50758853])

В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке.  
Таким артефактом будет отрицательное количество дней трудового стажа в столбце *days_employed*.  

#### Заменим все отрицательные значения в столбце положительными с помощью метода abs():

In [5]:
data['days_employed'] = data['days_employed'].abs()

#### Выведем медианное значение трудового стажа *days_employed* в днях для каждого типа занятости:

In [6]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) обнаружены аномально большие значения.  
Исправить такие значения сложно, поэтому оставим их как есть.

#### Выведем перечень уникальных значений столбца *children:*

In [7]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

В столбце *children* есть два аномальных значения. **Удалим** строки, в которых встречаются эти аномалии:

In [8]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

Выведем снова перечень уникальных значений столбца *children*, чтобы убедиться, что артефакты удалены:

In [9]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

### Удаление пропусков

#### Выведем количество пропущенных значений для каждого столбца:

In [10]:
data.isna().sum()

children               0
days_employed       2162
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2162
purpose                0
dtype: int64

Пропущенные значения обнаружены в двух столбцах: *days_employed* и *total_income*.  

Начнём работу со столбца **total_income**.  
На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу занятости из столбца *income_type*:

In [11]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

Убедимся, что все пропуски в столбце *income_type* заполнены:

In [12]:
data.isna().sum()

children               0
days_employed       2162
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64

Обработаем пропуски в столбце **days_employed**.  
Заполним их медианными значениями по каждому типу занятости из столбца *income_type:*

In [13]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

Убедимся, что все пропуски в столбце *days_employed* заполнены:

In [14]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

**Выводы:** В исходных данных присутствовали пропуски в двух столбцах:  
— days_employed  
— total_income  

Возможные причины пропусков:
- Заёмщик не указал о себе какие-либо сведения
- Система сбора информации при обработке заявки на кредит дала сбой
- Человеческий фактор: пропуски могли не заметить люди, принимающие заявку
- Иногда пропуски оставляют умышленно, рассчитывая на автозаполнение спецсимволами  

Заполнить пропуски медианным значением — лучшее решение в данной ситуации.  
В количественных переменных данные часто неоднородны, и некоторые значения могут сильно выделяться среди большинства. Медиана помогает избежать некорретных подсчетов для определения типичных значений выборки, так как ровно половина значений больше неё, а другая половина значений — меньше. 


### Изменение типов данных

Заменим вещественный тип данных в столбце *total_income* на целочисленный с помощью метода **astype():**

In [15]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

#### Выведем количество строк-дубликатов в данных.  
Если такие строки присутствуют, удалим их с помощью метода **drop_duplicates():**

In [16]:
# подсчет количества явных дубликатов:
data.duplicated().sum()

54

In [17]:
# удаление явных дубликатов:
data = data.drop_duplicates()

#### Обработаем неявные дубликаты в столбце *education*. 
В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв.   
Приведем их к нижнему регистру:

In [18]:
data['education'] = data['education'].str.lower()

### Категоризация данных

#### Разделим заёмщиков по категориям в зависимости от их ежемесячного дохода.  
Создадим новый столбец *total_income_category* и сохраним в него категорию на основании следующих диапазонов:   
- 0–30000 рублей — **E**
- 30001–50000 рублей — **D**
- 50001–200000 рублей — **C**
- 200001–1000000 рублей — **B**
- 1000001 рублей и выше — **A**

In [19]:
# напишем функцию, которая присвоит категории заёмщикам:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [20]:
# применим функцию к столбцу total_income и запишем категории в столбец total_income_category:
data['total_income_category'] = data['total_income'].apply(categorize_income)

#### Выведем перечень уникальных целей кредита из столбца *purpose:*

In [21]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

#### Разделим цели кредита на 4 основные категории:
- операции с автомобилем  
- операции с недвижимостью  
- проведение свадьбы  
- получение образования   

Далее запишем категорию в новый столбец *purpose_category*:

In [22]:
# напишем функцию, которая разделит цели кредита по категориям:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [23]:
# применим функцию к столбцу purpose и запишем категории в столбец purpose_category:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

## Исследовательский анализ данных

Ответим на ряд вопросов, позволяющих оценить надежность заёмщиков:

### 3.1 Есть ли зависимость между количеством детей и возвратом кредита в срок?

#### **3.1.1** Сделаем группировку данных должников по количеству детей в семье:

In [24]:
children_factor = data.groupby('children')['debt'].sum()
children_factor

children
0    1063
1     444
2     194
3      27
4       4
5       0
Name: debt, dtype: int64

**Вывод:** Предположительно, больше всего просрочек по кредиту у группы людей без детей. Но эти цифры не показательные, так как мы не знаем, какую долю от общего количества людей в группе занимают люди с просрочкой.  

#### **3.1.2** Посчитаем общее количество людей в каждой группе:

In [25]:
total_people = data.groupby('children')['debt'].count()
total_people

children
0    14107
1     4809
2     2052
3      330
4       41
5        9
Name: debt, dtype: int64

**Вывод:** Группа без детей является самой многочисленной, поэтому и людей с просрочкой по кредиту в ней больше.  
В группах с 3, 4 и 5 детьми очень мало человек по сравнению с другими. Есть смысл объединить их в одну общую группу - **"3 и более детей"**.  

#### **3.1.3** Объединим людей с количеством детей 3, 4 и 5 в общую группу - "3 и более детей":

In [26]:
total_people['3+'] = total_people[3] + total_people[4] + total_people[5]
total_people.drop(index=[3,4,5], inplace=True)
total_people

children
0     14107
1      4809
2      2052
3+      380
Name: debt, dtype: int64

#### **3.1.4** Посчитаем количество людей с просрочкой по кредиту в новой группе "3 и более детей":

In [27]:
children_factor['3+'] = children_factor[3] + children_factor[4] + children_factor[5]
children_factor.drop(index=[3,4,5], inplace=True)
children_factor

children
0     1063
1      444
2      194
3+      31
Name: debt, dtype: int64

#### **3.1.5** Посчитаем долю должников от общего количества людей в группе для каждой из групп:

In [28]:
share_of_debt = (children_factor / total_people)*100
round(share_of_debt, 1)

children
0     7.5
1     9.2
2     9.5
3+    8.2
Name: debt, dtype: float64

#### **3.1.6** Сделаем сводную таблицу по всем полученным данными:

In [29]:
data_debt = total_people.to_frame().merge(children_factor, on='children', how='left')
data_debt['share_of_debt'] = round((children_factor / total_people)*100, 1)

# переименуем столбцы в итоговой таблице:
data_debt.columns = ['total_people', 'people_with_debt', 'share_of_debt']

# применим сортировку по доле должников (от большей к меньшей):
data_debt.sort_values(by='share_of_debt', ascending=False)

,total_people,people_with_debt,share_of_debt
children,,,
2,2052,194,9.5
1,4809,444,9.2
3+,380,31,8.2
0,14107,1063,7.5


**Выводы:**  
- Доля должников в каждой из групп составляет от 7 до 10%  
- Самый **большой** процент людей с просрочкой в семьях, у которых есть **один** (9.2%) и **двое** детей (9.5%)
- Самый **низкий** процент должников у людей, у которых **нет детей:** 7.5%
- В семьях с количеством детей больше трех, должников **меньше**, чем у семей с одним или двумя детьми: всего 8.2%  

Подводя итог, можно сделать вывод, что зависимость между количеством детей и возвратом кредита в срок **отсутствует.**  
Несмотря на увеличение на 0.5% между должниками в семьях с одним ребенком и двумя детьми, тенденция не сохраняется: в категории, где детей более трех должников всего 8.2%  

При этом, хочется отметить, что у людей без детей самые низкие показатели по просрочкам по сравнению с другими группами.  
Таким образом, можно сделать вывод, что **наличие детей влияет** на факт просрочки по кредиту, а их **количество - нет.**

### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

#### **3.2.1** Посчитаем количество должников по каждому статусу семейного положения, представленного в таблице: 

In [30]:
family_pivot = data.pivot_table(index=['family_status'], values='debt', aggfunc='sum')
family_pivot.sort_values(by='debt', ascending=False)

,debt
family_status,
женат / замужем,927
гражданский брак,385
Не женат / не замужем,273
в разводе,84
вдовец / вдова,63


**Вывод:** Предположительно, больше всего просрочек по кредиту у людей, состоящих в браке.  
Но эти цифры не показательные, так как мы не знаем, какую долю от общего количества людей в группе занимают люди с просрочкой.  

#### **3.2.2** Посчитаем общее количество людей в каждом статусе:

In [31]:
family_pivot['total_people'] = data.groupby('family_status')['debt'].count() 
family_pivot.sort_values(by='debt', ascending=False)

,debt,total_people
family_status,,
женат / замужем,927,12266
гражданский брак,385,4146
Не женат / не замужем,273,2796
в разводе,84,1189
вдовец / вдова,63,951


**Вывод:** Группа людей, состоящих в браке, является самой многочисленной, поэтому и людей с просрочкой по кредиту в ней больше.  

#### **3.2.3** Посчитаем долю должников от общего количества людей в группе для каждого статуса:

In [32]:
family_pivot['share_of_debt'] = round((family_pivot['debt'] / family_pivot['total_people'])*100, 1)
family_pivot.sort_values(by='share_of_debt', ascending=False)

,debt,total_people,share_of_debt
family_status,,,
Не женат / не замужем,273,2796,9.8
гражданский брак,385,4146,9.3
женат / замужем,927,12266,7.6
в разводе,84,1189,7.1
вдовец / вдова,63,951,6.6


**Выводы:**  
- Доля должников в каждой из групп составляет от 6 до 10%
- Самый **большой** процент людей с просрочкой среди людей **не состоящих в браке** (9.8%) и людей живущих **гражданским** браком (9.3%)
- У людей в браке и людей в разводе схожие показатели: ≈ 7-7,5%
- Самые **низкие** показатели по должникам у **вдов и вдовцов:** 6.6%  

Зависимость между семейным положением и возвратом кредита в срок **есть:**  
люди, никогда не состоящие в браке и люди, живущие гражданским браком имеют в среднем на 3% **выше** показатели по просрочкам в сравнении с людьми, которые состоят в браке, разведены или овдовели.

### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Ранее мы разделили людей по категориям в зависимости от суммы дохода:
- E - люди, зарабатывающие до 30000 рублей
- D - люди, зарабатывающие от 30000 до 50000 рублей
- С - люди, зарабатывающие от 50000 до 200000 рублей
- B - люди, зарабатывающие от 200000 до 1 миллиона рублей
- A - люди, зарабатывающие больше 1 миллиона рублей  

#### **3.3.1** Проверим, сколько людей в каждой из категорий:

In [33]:
data['total_income_category'].value_counts()

C    15938
B     5014
D      349
A       25
E       22
Name: total_income_category, dtype: int64

**Вывод:** Больше всего людей в категории **C** - люди, зарабатывающие от 50000 до 200000 рублей.  

#### **3.3.2** Упростим категоризацию, разделив заёмщиков на 4 группы по размеру дохода:

In [34]:
# разделим людей на 4 равные группы с помощью метода qcut():
pd.qcut(data['total_income'], 4)

0        (195788.5, 2265604.0]
1         (107559.5, 142587.0]
2         (142587.0, 195788.5]
3        (195788.5, 2265604.0]
4         (142587.0, 195788.5]
                 ...          
21520    (195788.5, 2265604.0]
21521     (142587.0, 195788.5]
21522    (20666.999, 107559.5]
21523    (195788.5, 2265604.0]
21524    (20666.999, 107559.5]
Name: total_income, Length: 21348, dtype: category
Categories (4, interval[float64]): [(20666.999, 107559.5] < (107559.5, 142587.0] < (142587.0, 195788.5] < (195788.5, 2265604.0]]

Итого мы получили 4 равных группы по уровню дохода:
- менее 107 559.5 рублей
- от 107 559.5 до 142 587 рублей
- от 142 587 до 195 788.5 рублей 
- от 195 788.5 рублей

In [35]:
# напишем функцию, которая упростит категоризацию:
def simple_category(income):
        if income < 107559.5:
            return 'низкий доход'
        elif 107559.5 < income < 142587:
            return 'средний доход'
        elif 142587 < income < 195788.5:
            return 'высокий доход'
        elif income > 195788.5:
            return 'очень высокий доход'

data['simple_category'] = data['total_income'].apply(simple_category)
data['simple_category'].value_counts()

очень высокий доход    5337
низкий доход           5337
высокий доход          5213
средний доход          4387
Name: simple_category, dtype: int64

#### **3.3.3** Посчитаем, сколько должников в каждой из категорий:

In [36]:
income_pivot = data.pivot_table(index=['simple_category'], values='debt', aggfunc='sum')
income_pivot.sort_values(by='debt', ascending=False)

,debt
simple_category,
высокий доход,444
низкий доход,427
средний доход,382
очень высокий доход,381


**Вывод:** Предположительно, больше всего просрочек по кредиту у людей со высоким доходом.  
Но мы не знаем, какую долю от общего количества людей в группе занимают должники.  

#### **3.3.4** Посчитаем долю должников для каждой группы и объединим всю информацию в сводную таблицу:

In [37]:
income_pivot['total_people'] = data.groupby('simple_category')['debt'].count() 
income_pivot['share_of_debt'] = round(((income_pivot['debt'] / income_pivot['total_people']) * 100),1)
income_pivot.sort_values(by='share_of_debt', ascending=False)

,debt,total_people,share_of_debt
simple_category,,,
средний доход,382,4387,8.7
высокий доход,444,5213,8.5
низкий доход,427,5337,8.0
очень высокий доход,381,5337,7.1


**Выводы:**
- Доля должников в каждой из групп составляет от 7 до 9%
- Лидером среди **должников** является группа людей со **средним** доходом: 8.7%
- Самые **низкие** показатели по должникам у людей с **очень высоким** доходом: 7.1%  

Предположение о том, что больше всего просрочек по кредиту у людей с высоким доходом, не подтвердилось.  

Несмотря на это, зависимость между уровнем дохода и возвратом кредита в срок **отсутствует**:  
люди с **низким** доходом имеют на *0.7%* **меньше** просрочек по кредитам по сравнению людьми со средним доходом, и на *0.5%* меньше по сравнению с людьми с высоким доходом.

### 3.4 Как разные цели кредита влияют на его возврат в срок?

Ранее мы разделили цели кредита на 4 категории:
- операции с недвижимостью
- операции с автомобилем
- получение образования
- проведение свадьбы  

#### **3.4.1** Проверим, сколько должников в каждой из категорий:

In [38]:
purpose_pivot = data.pivot_table(index=['purpose_category'], values='debt', aggfunc='sum')
purpose_pivot.sort_values(by='debt', ascending=False)

,debt
purpose_category,
операции с недвижимостью,780
операции с автомобилем,400
получение образования,369
проведение свадьбы,183


**Вывод:** Предположительно, больше всего просрочек у людей, взявших кредит для операций с недвижимостью.  
Но эти цифры не показательные, так как мы не знаем, какую долю от общего количества людей в группе занимают люди с просрочкой.  

#### **3.4.2** Посчитаем общее количество людей в каждой категории:

In [39]:
purpose_pivot['total_people'] = data.groupby('purpose_category')['debt'].count() 
purpose_pivot

,debt,total_people
purpose_category,,
операции с автомобилем,400,4281
операции с недвижимостью,780,10754
получение образования,369,3989
проведение свадьбы,183,2324


#### **3.4.3** Посчитаем долю должников для каждой группы:

In [40]:
purpose_pivot['share_of_debt'] = round(((purpose_pivot['debt'] / purpose_pivot['total_people']) * 100),1)
purpose_pivot.sort_values(by='share_of_debt', ascending=False)

,debt,total_people,share_of_debt
purpose_category,,,
операции с автомобилем,400,4281,9.3
получение образования,369,3989,9.3
проведение свадьбы,183,2324,7.9
операции с недвижимостью,780,10754,7.3


**Выводы:**
- Доля должников в каждой из групп составляет от 7 до 9%
- Самый **большой** процент людей с просрочкой среди людей, взявших кредиты на **автомобиль** и на **образование:** 9.3%
- Самые **низкие** показатели по должникам у людей, взявших кредит для операций с **недвижимостью:** 7.3%  

Предположение о том, что больше всего просрочек по кредиту у людей, взявших кредит для операций с недвижимостью, не подтвердилось.  
Наоборот, у данной категории самые низкие показатели по должникам.

Люди, взявшие кредит на проведение свадьбы, находятся посередине: их показатели составляют 7.9%.

## Общий вывод

Исследование проходило в три основных этапа:    

**1. Обзор данных:**  
- Получена общая информация о датафрейме
- Сформировано первое представление об исходных данных  

**2. Предобработка данных:**  
- Устранены аномальные значения
- Заполнены пропуски в данных
- Устранены явные и неявные дубликаты
- Изменены типы данных в столбцах, где это необходимо  
- Присвоены категории заёмщикам в зависимости от их ежемесячного дохода  
- Выделены 4 основные группы в зависимости от целей кредита

**3. Исследовательский анализ данных:**
- Изучено влияние семейного положения, уровня дохода и количества детей клиента на факт погашения кредита в срок  



Целью проекта было исследование надежности заемщиков, а также ответы на следующие вопросы:  
—  Есть ли зависимость между количеством детей и возвратом кредита в срок?  
—  Есть ли зависимость между семейным положением и возвратом кредита в срок?  
—  Есть ли зависимость между уровнем дохода и возвратом кредита в срок?  
—  Как разные цели кредита влияют на его возврат в срок?   
 
Изучив статистику о платёжеспособности клиентов, были получены следующие **результаты:**  

- Количество детей **не влияет** на возврат кредита в срок: в семьях, где детей более трех процент должников на 1.3% меньше, чем в семьях с двумя детьми и на 1% меньше, чем в семьях с одним ребенком 

  
- Наличие детей **влияет** на факт просрочки по кредиту: у людей без детей самые низкие показатели по просрочкам по сравнению с семьями с детьми (на 1.7%, 2% и 0.7% меньше, чем у семей с 1, 2, 3 детьми соответственно)  


- Семейное положение **влияет** на возврат кредита в срок: люди, никогда не состоящие в браке и люди, живущие гражданским браком имеют в среднем на 3% выше показатели по просрочкам в сравнении с людьми, которые состоят в браке, разведены или овдовели  


- Уровнень дохода **не влияет** на возврат кредита в срок: люди с низким доходом имеют на *0.7%* меньше просрочек по кредитам, чем люди со средним доходом, и на *0.5%* меньше, чем люди с высоким доходом


- Самыми **надежными** заемщиками являются люди, взявшие кредиты для операций с недвижимостью: у них самый низкий процент просрочек:  
— на 2% ниже, чем у людей, взявших кредиты на автомобиль и на образование  
— на 0.6% ниже, чем у людей, взявших кредиты проведение свадьбы


- Самыми **ненадежными** заемщиками являются люди, взявшие кредиты на автомобиль и на образование: они чаще других имеют просрочки по кредитам:  
— на 2% выше, чем у людей, взявших кредиты на автомобиль и на образование  
— на 1.4% выше, чем у людей, взявших кредиты проведение свадьбы